In [5]:
from langchain_community.chat_models.fake import FakeListChatModel
from langchain_openai import ChatOpenAI

from agential.cog.lats.agent import LATSAgent
from agential.cog.fewshots.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_REACT
from agential.cog.lats.prompts import (
    HOTPOTQA_FEWSHOT_EXAMPLES_LATS_REFLECT,
    LATS_INSTRUCTION_HOTPOTQA,
    LATS_REFLECT_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_LATS_VALUE, 
    LATS_VALUE_INSTRUCTION_HOTPOTQA,
)

question = "What's the capital of France?"
key = "France"

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
)
agent = LATSAgent(llm=llm, benchmark="hotpotqa", n_samples=2, depth_limit=5)
agent.strategy.docstore.search = (
    lambda x: "Badr Hari is the best kick boxer in the world."
)

best_node, out = agent.generate(
    question=question,
    key=key,
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_LATS_REFLECT,
    value_examples=HOTPOTQA_FEWSHOT_EXAMPLES_LATS_VALUE,
    prompt=LATS_INSTRUCTION_HOTPOTQA,
    reflect_prompt=LATS_REFLECT_INSTRUCTION_HOTPOTQA,
    value_prompt=LATS_VALUE_INSTRUCTION_HOTPOTQA,
    additional_keys={},
    reflect_additional_keys={},
    value_additional_keys={},
    max_iterations=1,
    reset=True,
)

<PROMPT AGENT=====================================================================>
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the last passage successfully found by Search.
(3) Finish[answer], which returns the answer and finishes the task.

Here are some examples:
Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action 1: Search[Colorado orogeny]
Observation 1: The Colorado orogeny was an episode of m

In [7]:
question = "What's the capital of France?"
key = "Paris"

best_node, out = agent.generate(
    question=question,
    key=key,
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_LATS_REFLECT,
    value_examples=HOTPOTQA_FEWSHOT_EXAMPLES_LATS_VALUE,
    prompt=LATS_INSTRUCTION_HOTPOTQA,
    reflect_prompt=LATS_REFLECT_INSTRUCTION_HOTPOTQA,
    value_prompt=LATS_VALUE_INSTRUCTION_HOTPOTQA,
    additional_keys={},
    reflect_additional_keys={},
    value_additional_keys={},
    max_iterations=1,
    reset=False,
)

<PROMPT REFLECTION=====================================================================>
You are an advanced reasoning agent that can improve based on self refection. You will be given a previous reasoning trial in which you were given access to an Docstore API environment and a question to answer. You were unsuccessful in answering the question either because you guessed the wrong answer with Finish[<answer>], or you used up your set number of reasoning steps. In a few sentences, Diagnose a possible reason for failure and devise a new, concise, high level plan that aims to mitigate the same failure. Use complete sentences.  
Here are some examples:
Previous Trial:
Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought 1: The Colorado orogeny is a mountain-building event in the western United States, so the elevation range should be quite high.
Action 1: Search[eastern sector of the Colorado orogeny]
Observation 1: The C

In [9]:
out

[LATSOutput(iteration=0, current_node={'state': ReActOutput(thought='', action_type='', query='', observation='', answer='', external_tool_info={}), 'visits': 0, 'value': 0, 'depth': 0, 'is_terminal': False, 'reward': 0}, children_nodes=[{'state': ReActOutput(thought='I need to search for the capital of France.', action_type='Search', query='capital of France', observation='Badr Hari is the best kick boxer in the world.', answer='', external_tool_info={'search_result': 'Badr Hari is the best kick boxer in the world.', 'lookup_result': ''}), 'visits': 0, 'value': 0.0, 'depth': 1, 'is_terminal': False, 'reward': 0}], values=[{'node_idx': 0, 'explanation': 'Explanation not found', 'value': 0.0}], simulation_reward=-1.0, simulation_terminal_node={'state': ReActOutput(thought='The search results are not helpful. I should try a different search engine or source to find the answer to the question.', action_type='Search', query='What is the capital of France', observation='Badr Hari is the bes

In [13]:
agent.strategy.value_cache

{'\nThought 1: I need to search for the capital of France.\nAction 1: Search[capital of France]\nObservation 1: Badr Hari is the best kick boxer in the world.::': 'This trajectory is incorrect because the search result for "capital of France" returned information about a kickboxer, which is unrelated to the question. The action taken did not lead to relevant information regarding the capital of France. \nCorrectness score: 1',
 "\nThought 1: I need to search for the capital of France.\nAction 1: Search[capital of France]\nObservation 1: Badr Hari is the best kick boxer in the world.::Question: What's the capital of France?\n\nThought 1: I need to search for the capital of France.\nAction 1: Search[capital of France]\nObservation 1: Badr Hari is the best kick boxer in the world.\nThought 2: The search results are irrelevant. I need to search again for the capital of France.\nAction 2: Search[capital of France]\nObservation 2: Badr Hari is the best kick boxer in the world.\nThought 3: Th